In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

Overview.

In [3]:
matches = pd.read_csv('understat.csv')

In [4]:
matches.head()

,Unnamed: 0,home_team,away_team,date,league,season,home_score,away_score,home_xg,away_xg,home_player_name_1,home_player_position_1,home_player_name_2,home_player_position_2,home_player_name_3,home_player_position_3,home_player_name_4,home_player_position_4,home_player_name_5,home_player_position_5,home_player_name_6,home_player_position_6,home_player_name_7,home_player_position_7,home_player_name_8,home_player_position_8,home_player_name_9,home_player_position_9,home_player_name_10,home_player_position_10,home_player_name_11,home_player_position_11,home_player_name_12,home_player_position_12,home_player_name_13,home_player_position_13,home_player_name_14,home_player_position_14,home_player_name_15,home_player_position_15,home_player_name_16,home_player_position_16,home_player_name_17,home_player_position_17,away_player_name_1,away_player_position_1,away_player_name_2,away_player_position_2,away_player_name_3,away_player_position_3,away_player_name_4,away_player_position_4,away_player_name_5,away_player_position_5,away_player_name_6,away_player_position_6,away_player_name_7,away_player_position_7,away_player_name_8,away_player_position_8,away_player_name_9,away_player_position_9,away_player_name_10,away_player_position_10,away_player_name_11,away_player_position_11,away_player_name_12,away_player_position_12,away_player_name_13,away_player_position_13,away_player_name_14,away_player_position_14,away_player_name_15,away_player_position_15,away_player_name_16,away_player_position_16,away_player_name_17,away_player_position_17,datetime
0,5179,Bayern Munich,Wolfsburg,August 22 2014,Bundesliga,2014/2015,2,1,2.57,1.20,Manuel Neuer,GK,Philipp Lahm,DC,Dante,DC,Holger Badstuber,DC,Arjen Robben,MR,Gianluca Gaudino,MC,Pierre-Emile Højbjerg,Sub,David Alaba,MC,Juan Bernat,ML,Mario Götze,AMC,Xherdan Shaqiri,Sub,Thomas Müller,FW,Robert Lewandowski,FW,Sebastian Rode,Sub,NaN,NaN,NaN,NaN,NaN,NaN,Max Grün,GK,Sebastian Jung,DR,Naldo,DC,Robin Knoche,DC,Ricardo Rodríguez,DL,Josuha Guilavogui,DMC,Luiz Gustavo,DMC,Junior Malanda,Sub,Vieirinha,AMR,Daniel Caligiuri,Sub,Aaron Hunt,AMC,Maximilian Arnold,Sub,Kevin De Bruyne,AML,Ivica Olic,FW,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-22
1,5180,Hoffenheim,Augsburg,August 23 2014,Bundesliga,2014/2015,2,0,1.53,0.28,Oliver Baumann,GK,Andreas Beck,DR,Niklas Süle,DC,Ermin Bicakcic,DC,Jin-Su Kim,DL,Eugen Polanski,DMC,Sebastian Rudy,DMC,Kevin Volland,AMR,Tobias Strobl,Sub,Roberto Firmino,AMC,Sven Schipplock,Sub,Tarik Elyounoussi,AML,Ádám Szalai,FW,Anthony Modeste,Sub,NaN,NaN,NaN,NaN,NaN,NaN,Marwin Hitz,GK,Paul Verhaegh,DR,Ragnar Klavan,DC,Jan-Ingwer Callsen-Bracker,DC,Abdul Rahman Baba,DL,Daniel Baier,DMC,Markus Feulner,DMC,Caiuby,AMR,Shawn Parker,Sub,Halil Altintop,AMC,Nikola Djurdjic,Sub,Tobias Werner,AML,Tim Matavz,FW,Sascha Mölders,Sub,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-23
2,5181,Hannover 96,Schalke 04,August 23 2014,Bundesliga,2014/2015,2,1,1.18,0.96,Ron-Robert Zieler,GK,Hiroki Sakai,DR,Christian Schulz,DC,Marcelo,DC,Christian Pander,DL,Leonardo Bittencourt,MR,Ceyhun Gülselam,Sub,Leon Andreasen,MC,Manuel Schmiedebach,MC,Miiko Albornoz,ML,Hiroshi Kiyotake,Sub,Edgar Prib,FW,Joselu,FW,Artur Sobiech,Sub,NaN,NaN,NaN,NaN,NaN,NaN,Ralf Fährmann,GK,Kaan Ayhan,DR,Joel Matip,DC,Benedikt Höwedes,DC,Sead Kolasinac,DL,Christian Fuchs,Sub,Roman Neustädter,DMC,Kevin-Prince Boateng,DMC,Sidney Sam,AMR,Christian Clemens,Sub,Julian Draxler,AMC,Max Meyer,Sub,Eric Maxim Choupo-Moting,AML,Klaas-Jan Huntelaar,FW,NaN,NaN,NaN,NaN,NaN,NaN,2014-08-23
3,5182,Hertha Berlin,Werder Bremen,August 23 2014,Bundesliga,2014/2015,2,2,1.76,1.19,Thomas Kraft,GK,Peter Pekarík,DR,John Heitinga,DC,Sebastian Langkamp,DC,Nico Schulz,DL,Hajime Hosogai,DMC,Peter Niemeyer,DMC,Jens Hegeler,Sub,Roy Beerens,AMR,Ronny,AMC,Sandro Wagner,Sub,Genki Haraguchi,AML,Julian Schieber,FW,Sami Allagui,Sub,NaN,NaN,NaN,NaN,NaN,NaN,Raphael Wolf,GK,Clemens Fritz,DR,Assani Lukimya,DC,Luca Caldirola,DC,Santiago García,DL,Felix Kroos,DMC,Davie Selke,Sub,Zlatko Junuzovic,MC,Cedrick Makiadi,MC

In [5]:
ratings = pd.concat([
    pd.read_csv('England-Premier-League_ratings.csv'),
    pd.read_csv('Spain-LaLiga_ratings.csv'),
    pd.read_csv('Italy-Serie-A_ratings.csv'),
    pd.read_csv('Germany-Bundesliga_ratings.csv'),
    pd.read_csv('France-Ligue-1_ratings.csv')
], axis=0)

In [6]:
ratings.head()

,Unnamed: 0,match_url,player,position,rating,home_team,away_team
0,0,https://1xbet.whoscored.com/Matches/829783/Liv...,Boaz Myhill,GK,7.43,West Bromwich Albion,Chelsea
1,1,https://1xbet.whoscored.com/Matches/829783/Liv...,Craig Dawson,DR,7.92,West Bromwich Albion,Chelsea
2,2,https://1xbet.whoscored.com/Matches/829783/Liv...,Jonas Olsson,DC,6.85,West Bromwich Albion,Chelsea
3,3,https://1xbet.whoscored.com/Matches/829783/Liv...,Gareth McAuley,DC,6.79,West Bromwich Albion,Chelsea
4,4,https://1xbet.whoscored.com/Matches/829783/Liv...,Joleon Lescott,DL,7.27,West Bromwich Albion,Chelsea


In [7]:
ratings = ratings.drop(columns=['Unnamed: 0'])

Add some more ratings.

In [8]:
def data_prettify(data_raw, teams_file_name):
    data = pd.DataFrame()

    data['match_url'] = data_raw['Original_URL']
    data['player'] = data_raw['Title']
    data['position'] = data_raw['col12lg21']
    data['rating'] = data_raw['rating']

    data['rating'] = data['rating'].apply(lambda x: np.nan if pd.isna(x) or x == '-' else float(str(x)))
    data['position'] = data['position'].apply(lambda x: np.nan if pd.isna(x) else x[2:].strip())

    teams = set()
    file = open(teams_file_name, 'r')
    for line in file:
        teams.add(line.strip())
    teams.remove('P')

    home_team = []
    away_team = []
    for i in range(len(data)):
        url = data.iloc[i]['match_url']
        home, away = np.nan, np.nan
        for team in teams:
            index = url.find(team.replace(' ', '-'))
            if index >= 0:
                if index + len(team) == len(url):
                    away_team.append(team)
                else:
                    home_team.append(team)

    data['home_team'] = home_team
    data['away_team'] = away_team

    return data

In [9]:
data_raw = pd.read_csv('add_ratings.csv')

In [10]:
file = 'add_ratings_teams.txt'

In [11]:
data = data_prettify(data_raw, file)

In [12]:
data.head()

,match_url,player,position,rating,home_team,away_team
0,https://1xbet.whoscored.com/Matches/1640794/Li...,Emiliano Martínez,GK,7.87,Aston Villa,Brentford
1,https://1xbet.whoscored.com/Matches/1640794/Li...,Matty Cash,DR,7.17,Aston Villa,Brentford
2,https://1xbet.whoscored.com/Matches/1640794/Li...,Tyrone Mings,DC,7.54,Aston Villa,Brentford
3,https://1xbet.whoscored.com/Matches/1640794/Li...,Ezri Konsa,DC,7.77,Aston Villa,Brentford
4,https://1xbet.whoscored.com/Matches/1640794/Li...,Ashley Young,DL,7.46,Aston Villa,Brentford


In [13]:
ratings = pd.concat([
    ratings,
    data
], axis=0)

In [14]:
len(ratings)

640774

Extract the season.

In [15]:
def get_season(match_url):
    leagues = [
        'England-Premier-League',
        'Spain-LaLiga',
        'Italy-Serie-A',
        'Germany-Bundesliga',
        'France-Ligue-1'
    ]

    start = 0
    for L in leagues:
        pos = match_url.find(L)
        if pos >= 0:
            start = pos + len(L) + 1
            break

    S = match_url[start:start + 9]
    S = S.replace('-', '/')
    return S

In [16]:
ratings['season'] = ratings['match_url'].apply(get_season)

In [17]:
ratings.head()

,match_url,player,position,rating,home_team,away_team,season
0,https://1xbet.whoscored.com/Matches/829783/Liv...,Boaz Myhill,GK,7.43,West Bromwich Albion,Chelsea,2014/2015
1,https://1xbet.whoscored.com/Matches/829783/Liv...,Craig Dawson,DR,7.92,West Bromwich Albion,Chelsea,2014/2015
2,https://1xbet.whoscored.com/Matches/829783/Liv...,Jonas Olsson,DC,6.85,West Bromwich Albion,Chelsea,2014/2015
3,https://1xbet.whoscored.com/Matches/829783/Liv...,Gareth McAuley,DC,6.79,West Bromwich Albion,Chelsea,2014/2015
4,https://1xbet.whoscored.com/Matches/829783/Liv...,Joleon Lescott,DL,7.27,West Bromwich Albion,Chelsea,2014/2015


In [18]:
ratings['season'].unique()

array(['2014/2015', '2015/2016', '2016/2017', '2017/2018', '2018/2019',
       '2019/2020', '2020/2021', '2021/2022', '2022/2023'], dtype=object)

Matching team names.

In [19]:
teams_matches = sorted(list(matches['home_team'].unique()))
print(teams_matches)

['AC Milan', 'Ajaccio', 'Alaves', 'Almeria', 'Amiens', 'Angers', 'Arminia Bielefeld', 'Arsenal', 'Aston Villa', 'Atalanta', 'Athletic Club', 'Atletico Madrid', 'Augsburg', 'Auxerre', 'Barcelona', 'Bayer Leverkusen', 'Bayern Munich', 'Benevento', 'Bochum', 'Bologna', 'Bordeaux', 'Borussia Dortmund', 'Borussia M.Gladbach', 'Bournemouth', 'Brentford', 'Brescia', 'Brest', 'Brighton', 'Burnley', 'Cadiz', 'Caen', 'Cagliari', 'Cardiff', 'Carpi', 'Celta Vigo', 'Cesena', 'Chelsea', 'Chievo', 'Clermont Foot', 'Cordoba', 'Cremonese', 'Crotone', 'Crystal Palace', 'Darmstadt', 'Deportivo La Coruna', 'Dijon', 'Eibar', 'Eintracht Frankfurt', 'Elche', 'Empoli', 'Espanyol', 'Everton', 'Evian Thonon Gaillard', 'FC Cologne', 'Fiorentina', 'Fortuna Duesseldorf', 'Freiburg', 'Frosinone', 'Fulham', 'GFC Ajaccio', 'Genoa', 'Getafe', 'Girona', 'Granada', 'Greuther Fuerth', 'Guingamp', 'Hamburger SV', 'Hannover 96', 'Hertha Berlin', 'Hoffenheim', 'Huddersfield', 'Hull', 'Ingolstadt', 'Inter', 'Juventus', 'Las 

In [20]:
teams_ratings = sorted(list(ratings['home_team'].unique()))
print(teams_ratings)

['AC Ajaccio', 'AC Carpi', 'AC Milan', 'Almeria', 'Amiens', 'Angers', 'Arminia Bielefeld', 'Arsenal', 'Aston Villa', 'Atalanta', 'Athletic Bilbao', 'Atletico Madrid', 'Augsburg', 'Auxerre', 'Barcelona', 'Bayer Leverkusen', 'Bayern Munich', 'Benevento', 'Bochum', 'Bologna', 'Bordeaux', 'Borussia Dortmund', 'Borussia M Gladbach', 'Bournemouth', 'Brentford', 'Brescia', 'Brest', 'Brighton', 'Burnley', 'Cadiz', 'Caen', 'Cagliari', 'Cardiff', 'Celta Vigo', 'Cesena', 'Chelsea', 'Chievo', 'Clermont Foot', 'Cordoba', 'Cremonese', 'Crotone', 'Crystal Palace', 'Darmstadt', 'Deportivo Alaves', 'Deportivo La Coruna', 'Dijon', 'Eibar', 'Eintracht Frankfurt', 'Elche', 'Empoli', 'Espanyol', 'Everton', 'FC Koln', 'Fiorentina', 'Fortuna Duesseldorf', 'Freiburg', 'Frosinone', 'Fulham', 'GFC Ajaccio', 'Genoa', 'Getafe', 'Girona', 'Granada', 'Greuther Fuerth', 'Guingamp', 'Hamburger SV', 'Hannover 96', 'Hertha Berlin', 'Hoffenheim', 'Huddersfield', 'Hull', 'Ingolstadt', 'Inter', 'Juventus', 'Las Palmas', '

In [21]:
teams_match_dict = {
    'AC Ajaccio': 'Ajaccio',
    'AC Carpi': 'Carpi',
    'Athletic Bilbao': 'Athletic Club',
    'Borussia M Gladbach': 'Borussia M.Gladbach',
    'Deportivo Alaves': 'Alaves',
    'FC Koln': 'FC Cologne',
    'Newcastle': 'Newcastle United',
    'Paris Saint-Germain': 'Paris Saint Germain',
    'RB Leipzig': 'RasenBallsport Leipzig',
    'Thonon Evian Grand Geneve': 'Evian Thonon Gaillard',
    'Wolves': 'Wolverhampton Wanderers'
}

We also have to rename *Parma* to *Parma Calcio 1913* in **matches** dataset since it is the same club (the name has changed).

In [22]:
def rename_Parma(team):
    if team == 'Parma':
        return 'Parma Calcio 1913'
    return team

In [23]:
matches['home_team'] = matches['home_team'].apply(rename_Parma)
matches['away_team'] = matches['away_team'].apply(rename_Parma)

In [24]:
matches[np.logical_or(
    matches['home_team'] == 'Parma',
    matches['away_team'] == 'Parma'
)]

,Unnamed: 0,home_team,away_team,date,league,season,home_score,away_score,home_xg,away_xg,home_player_name_1,home_player_position_1,home_player_name_2,home_player_position_2,home_player_name_3,home_player_position_3,home_player_name_4,home_player_position_4,home_player_name_5,home_player_position_5,home_player_name_6,home_player_position_6,home_player_name_7,home_player_position_7,home_player_name_8,home_player_position_8,home_player_name_9,home_player_position_9,home_player_name_10,home_player_position_10,home_player_name_11,home_player_position_11,home_player_name_12,home_player_position_12,home_player_name_13,home_player_position_13,home_player_name_14,home_player_position_14,home_player_name_15,home_player_position_15,home_player_name_16,home_player_position_16,home_player_name_17,home_player_position_17,away_player_name_1,away_player_position_1,away_player_name_2,away_player_position_2,away_player_name_3,away_player_position_3,away_player_name_4,away_player_position_4,away_player_name_5,away_player_position_5,away_player_name_6,away_player_position_6,away_player_name_7,away_player_position_7,away_player_name_8,away_player_position_8,away_player_name_9,away_player_position_9,away_player_name_10,away_player_position_10,away_player_name_11,away_player_position_11,away_player_name_12,away_player_position_12,away_player_name_13,away_player_position_13,away_player_name_14,away_player_position_14,away_player_name_15,away_player_position_15,away_player_name_16,away_player_position_16,away_player_name_17,away_player_position_17,datetime


In [25]:
def rename_teams(team):
    teams_match_dict = {
        'AC Ajaccio': 'Ajaccio',
        'AC Carpi': 'Carpi',
        'Athletic Bilbao': 'Athletic Club',
        'Borussia M Gladbach': 'Borussia M.Gladbach',
        'Deportivo Alaves': 'Alaves',
        'FC Koln': 'FC Cologne',
        'Newcastle': 'Newcastle United',
        'Paris Saint-Germain': 'Paris Saint Germain',
        'RB Leipzig': 'RasenBallsport Leipzig',
        'Thonon Evian Grand Geneve': 'Evian Thonon Gaillard',
        'Wolves': 'Wolverhampton Wanderers'
    }

    if team in teams_match_dict:
        return teams_match_dict[team]
    return team

In [26]:
ratings['home_team'] = ratings['home_team'].apply(rename_teams)
ratings['away_team'] = ratings['away_team'].apply(rename_teams)

In [27]:
teams_matches = sorted(set(list(matches['home_team'].unique()) + list(matches['away_team'].unique())))
teams_ratings = sorted(set(list(ratings['home_team'].unique()) + list(ratings['away_team'].unique())))

In [28]:
teams_matches == teams_ratings

True

Some cosmetic procedures.

In [29]:
ratings = ratings.dropna(subset=['player', 'position'])

In [30]:
def swap_rows(df, row1, row2):
    df.iloc[row1], df.iloc[row2] =  df.iloc[row2].copy(), df.iloc[row1].copy()
    return df

In [31]:
N = len(ratings)
for i in range(N - 1):
    pos_1, rate_1 = ratings.iloc[i]['position'], ratings.iloc[i]['rating']
    pos_2, rate_2 = ratings.iloc[i + 1]['position'], ratings.iloc[i + 1]['rating']

    if (pos_1 == 'GK') and (not pd.isna(rate_1)) and (pos_2 == 'Sub') and (pd.isna(rate_2)):
        ratings = swap_rows(ratings, i, i + 1)

In [32]:
rows = dict()
for i in range(44251, 44269):
    rows[i] = ratings.iloc[i].copy()

ratings.iloc[44251] = rows[44260]
ratings.iloc[44252] = rows[44264]
ratings.iloc[44253] = rows[44251]
ratings.iloc[44254] = rows[44261]
ratings.iloc[44255] = rows[44265]
ratings.iloc[44256] = rows[44254]
ratings.iloc[44257] = rows[44252]
ratings.iloc[44258] = rows[44266]
ratings.iloc[44259] = rows[44253]
ratings.iloc[44260] = rows[44262]
ratings.iloc[44261] = rows[44257]
ratings.iloc[44262] = rows[44255]
ratings.iloc[44263] = rows[44263]
ratings.iloc[44264] = rows[44267]
ratings.iloc[44265] = rows[44258]
ratings.iloc[44266] = rows[44256]
ratings.iloc[44267] = rows[44268]
ratings.iloc[44268] = rows[44259]

In [33]:
rows = dict()
for i in range(93302, 93320):
    rows[i] = ratings.iloc[i].copy()

ratings.iloc[93302] = rows[93303]
ratings.iloc[93303] = rows[93315]
ratings.iloc[93304] = rows[93302]
ratings.iloc[93305] = rows[93304]
ratings.iloc[93306] = rows[93316]
ratings.iloc[93307] = rows[93310]
ratings.iloc[93308] = rows[93305]
ratings.iloc[93309] = rows[93307]
ratings.iloc[93310] = rows[93317]
ratings.iloc[93311] = rows[93312]
ratings.iloc[93312] = rows[93306]
ratings.iloc[93313] = rows[93309]
ratings.iloc[93314] = rows[93318]
ratings.iloc[93315] = rows[93313]
ratings.iloc[93316] = rows[93308]
ratings.iloc[93317] = rows[93314]
ratings.iloc[93318] = rows[93319]
ratings.iloc[93319] = rows[93311]

In [34]:
rows = dict()
for i in range(72424, 72437):
    rows[i] = ratings.iloc[i].copy()

ratings.iloc[72424] = rows[72430]
ratings.iloc[72425] = rows[72431]
ratings.iloc[72426] = rows[72424]
ratings.iloc[72427] = rows[72425]
ratings.iloc[72428] = rows[72432]
ratings.iloc[72429] = rows[72434]
ratings.iloc[72430] = rows[72426]
ratings.iloc[72431] = rows[72427]
ratings.iloc[72432] = rows[72433]
ratings.iloc[72433] = rows[72428]
ratings.iloc[72434] = rows[72435]
ratings.iloc[72435] = rows[72436]
ratings.iloc[72436] = rows[72429]

In [35]:
rows = dict()
for i in range(239308, 239329):
    rows[i] = ratings.iloc[i].copy()

ratings.iloc[239308] = rows[239309]
ratings.iloc[239309] = rows[239317]
ratings.iloc[239310] = rows[239308]
ratings.iloc[239311] = rows[239322]
ratings.iloc[239312] = rows[239310]
ratings.iloc[239313] = rows[239318]
ratings.iloc[239314] = rows[239323]
ratings.iloc[239315] = rows[239312]
ratings.iloc[239316] = rows[239319]
ratings.iloc[239317] = rows[239311]
ratings.iloc[239318] = rows[239324]
ratings.iloc[239319] = rows[239315]
ratings.iloc[239320] = rows[239320]
ratings.iloc[239321] = rows[239313]
ratings.iloc[239322] = rows[239325]
ratings.iloc[239323] = rows[239316]
ratings.iloc[239324] = rows[239321]
ratings.iloc[239325] = rows[239314]
ratings.iloc[239326] = rows[239326]
ratings.iloc[239327] = rows[239328]
ratings.iloc[239328] = rows[239327]

In [36]:
rows = dict()
for i in range(265794, 265810):
    rows[i] = ratings.iloc[i].copy()

ratings.iloc[265794] = rows[265802]
ratings.iloc[265795] = rows[265795]
ratings.iloc[265796] = rows[265799]
ratings.iloc[265797] = rows[265794]
ratings.iloc[265798] = rows[265803]
ratings.iloc[265799] = rows[265796]
ratings.iloc[265800] = rows[265801]
ratings.iloc[265801] = rows[265804]
ratings.iloc[265802] = rows[265805]
ratings.iloc[265803] = rows[265806]
ratings.iloc[265804] = rows[265807]
ratings.iloc[265805] = rows[265808]
ratings.iloc[265806] = rows[265809]
ratings.iloc[265807] = rows[265797]
ratings.iloc[265808] = rows[265798]
ratings.iloc[265809] = rows[265800]

In [37]:
ratings = swap_rows(ratings, 19938, 19941)

In [38]:
ratings = swap_rows(ratings, 94779, 94780)

In [39]:
ratings = swap_rows(ratings, 457996, 457997)

In [40]:
ratings = swap_rows(ratings, 555590, 555592)

In [41]:
ratings = swap_rows(ratings, 567203, 567204)

In [42]:
ratings = swap_rows(ratings, 568679, 568680)

In [43]:
ratings = swap_rows(ratings, 105543, 105544)

In [44]:
ratings = swap_rows(ratings, 115963, 115965)

In [45]:
ratings = swap_rows(ratings, 486630, 486632)

In [46]:
ratings = swap_rows(ratings, 270081, 270082)

In [47]:
ratings = swap_rows(ratings, 274059, 274060)

In [48]:
ratings = swap_rows(ratings, 280059, 280060)

In [49]:
ratings = swap_rows(ratings, 340499, 340500)

In [50]:
matches_set = set()
for i in range(len(matches)):
    home_team = matches.iloc[i]['home_team']
    away_team = matches.iloc[i]['away_team']
    season = matches.iloc[i]['season']

    matches_set.add((home_team, away_team, season))

In [51]:
len(matches_set)

16332

In [52]:
len(ratings)

640741

In [53]:
ratings = ratings.drop_duplicates(keep='last')

In [54]:
len(ratings)

640501

In [64]:
matches_ratings_list = []

In [65]:
home_or_away = []
flag = False

prev_pos = 'Sub'

team_size = []
team_size_cnt = 0

for i in range(len(ratings)):
    pos = ratings.iloc[i]['position']
    if pos == 'GK' and prev_pos == 'Sub':
        flag = not flag
        team_size.append(team_size_cnt)
        team_size_cnt = 0

        home_team = ratings.iloc[i]['home_team']
        away_team = ratings.iloc[i]['away_team']
        season = ratings.iloc[i]['season']
        matches_ratings_list.append((home_team, away_team, season))

    team_size_cnt += 1
    prev_pos = pos

    if flag:
        home_or_away.append('home')
    else:
        home_or_away.append('away')

team_size.append(team_size_cnt)
team_size.remove(0)

ratings['home_or_away'] = home_or_away

In [66]:
print(f'Number of matches: {len(team_size) / 2}')
print(f'Number of players in teams: {sorted(set(team_size))}')

Number of matches: 16332.0
Number of players in teams: [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]


In [67]:
matches_ratings_set = set(matches_ratings_list)
len(matches_ratings_set), len(matches_ratings_list)

(16332, 32664)

In [68]:
matches_set - matches_ratings_set

set()

In [69]:
ratings.head()

,match_url,player,position,rating,home_team,away_team,season,home_or_away
0,https://1xbet.whoscored.com/Matches/829783/Liv...,Boaz Myhill,GK,7.43,West Bromwich Albion,Chelsea,2014/2015,home
1,https://1xbet.whoscored.com/Matches/829783/Liv...,Craig Dawson,DR,7.92,West Bromwich Albion,Chelsea,2014/2015,home
2,https://1xbet.whoscored.com/Matches/829783/Liv...,Jonas Olsson,DC,6.85,West Bromwich Albion,Chelsea,2014/2015,home
3,https://1xbet.whoscored.com/Matches/829783/Liv...,Gareth McAuley,DC,6.79,West Bromwich Albion,Chelsea,2014/2015,home
4,https://1xbet.whoscored.com/Matches/829783/Liv...,Joleon Lescott,DL,7.27,West Bromwich Albion,Chelsea,2014/2015,home


Adding datetime.

In [70]:
def add_datetime(row, matches):
    home_team = row['home_team']
    away_team = row['away_team']
    season = row['season']

    c_1 = (matches['home_team'] == home_team)
    c_2 = (matches['away_team'] == away_team)
    c_3 = (matches['season'] == season)

    tmp_df = matches[c_1 & c_2 & c_3]
    assert len(tmp_df) == 1

    dt = tmp_df.iloc[0]['datetime']
    return dt

In [78]:
ratings = ratings.sort_values(by=['season', 'home_team', 'away_team', 'home_or_away', 'position', 'rating', 'player'])

In [79]:
ratings.head(40)

,match_url,player,position,rating,home_team,away_team,season,home_or_away
8560,https://1xbet.whoscored.com/Matches/865957/Liv...,Guglielmo Stendardo,DC,7.54,AC Milan,Atalanta,2014/2015,away
8561,https://1xbet.whoscored.com/Matches/865957/Liv...,Giuseppe Biava,DC,7.91,AC Milan,Atalanta,2014/2015,away
8562,https://1xbet.whoscored.com/Matches/865957/Liv...,Cristiano Del Grosso,DL,7.81,AC Milan,Atalanta,2014/2015,away
8559,https://1xbet.whoscored.com/Matches/865957/Liv...,Yohan Benalouane,DR,7.43,AC Milan,Atalanta,2014/2015,away
8568,https://1xbet.whoscored.com/Matches/865957/Liv...,Mauricio Pinilla,FW,6.49,AC Milan,Atalanta,2014/2015,away
8567,https://1xbet.whoscored.com/Matches/865957/Liv...,Germán Denis,FW,7.23,AC Milan,Atalanta,2014/2015,away
8558,https://1xbet.whoscored.com/Matches/865957/Liv...,Marco Sportiello,GK,7.18,AC Milan,Atalanta,2014/2015,away
8564,https://1xbet.whoscored.com/Matches/865957/Liv...,Carlos Carmona,MC,6.78,AC Milan,Atalanta,2014/2015,away
8565,https://1xbet.whoscored.com/Matches/865957/Liv...,Luca Cigarini,MC,7.20,AC Milan,Atalanta,2014/2015,away
8566,https://1xbet.whoscored.com/Matches/865957/Liv...,Maximiliano Moralez,ML,7.29,AC Milan,Atalanta,2014/2015,away


In [80]:
dt_dict = dict()

for i in range(len(matches)):
    home_team = matches.iloc[i]['home_team']
    away_team = matches.iloc[i]['away_team']
    season = matches.iloc[i]['season']
    dt = matches.iloc[i]['datetime']

    dt_dict[(home_team, away_team, season)] = dt

In [81]:
len(dt_dict) == len(matches)

True

In [82]:
dt = []
S = '2014/2015'

for i in range(len(ratings)):
    home_team = ratings.iloc[i]['home_team']
    away_team = ratings.iloc[i]['away_team']
    season = ratings.iloc[i]['season']
    if season != S:
        print(f'season {S} done')
        S = season

    date = dt_dict[(home_team, away_team, season)]
    dt.append(date)

print(f'season {S} done')
ratings['datetime'] = dt

season 2014/2015 done
season 2015/2016 done
season 2016/2017 done
season 2017/2018 done
season 2018/2019 done
season 2019/2020 done
season 2020/2021 done
season 2021/2022 done
season 2022/2023 done


In [83]:
ratings.head()

,match_url,player,position,rating,home_team,away_team,season,home_or_away,datetime
8560,https://1xbet.whoscored.com/Matches/865957/Liv...,Guglielmo Stendardo,DC,7.54,AC Milan,Atalanta,2014/2015,away,2015-01-18
8561,https://1xbet.whoscored.com/Matches/865957/Liv...,Giuseppe Biava,DC,7.91,AC Milan,Atalanta,2014/2015,away,2015-01-18
8562,https://1xbet.whoscored.com/Matches/865957/Liv...,Cristiano Del Grosso,DL,7.81,AC Milan,Atalanta,2014/2015,away,2015-01-18
8559,https://1xbet.whoscored.com/Matches/865957/Liv...,Yohan Benalouane,DR,7.43,AC Milan,Atalanta,2014/2015,away,2015-01-18
8568,https://1xbet.whoscored.com/Matches/865957/Liv...,Mauricio Pinilla,FW,6.49,AC Milan,Atalanta,2014/2015,away,2015-01-18


In [84]:
ratings.to_csv('ratings.csv')

In [85]:
matches.to_csv('understat_upd.csv')